# data preprocess and fitness alignment

wt长度168
D G I
KPTENNEDRNIFAVVENFVTTDIDADRGKLPGKKLPLEVLKEMEANARPAGCTRGCLICLSKIKCTLKMKKFIPGRCHTYEGDKSTGQGGIGPAIVDIPEIPGFKDMEPMEQFIAQVDLCEDCTTGCLKGLANVQCSDLLKKWLPQRCATFASKIQGQVDKIKGAGGD
KPTENNKDFNIVAVASNFATTDLDADRGKLPGKKLPLEVLKEMEANARKAGCTRGCLICLSKIKCTLKMKKFIPGRCHTYEGDKSTGQGGIGPAIVDIPEIPGFKDGEPMEQFIAQVDLCVDCTTGCLKGLANVRCSDLLKKWLPDRCATFANKIQGEVDTIKGMNGD
KPTENNEDFNIVAVASNFATTDLDADRGKLPGKKLPLEVLKEMEANARKAGCTRGCLICLSKIKCTLKMKKFIPGRCHTYEGDKSTGQGGIGPAIVDIPEIPGFKDIEPMEQFIAQVDLCVDCTTGCLKGLANVHCSDLLKKWLPDRCASFATKIQKQVDKIKGMAGD
KPTENNEDFNIVAVASNFATTDLDADRGKLPGKKLPLEVLKEMEANARKAGCTRGCLICLSKIKCTLKMKKFIPGRCHTYEGDKSTGQGGIGPAIVDIPEIPGFKDMEPMEQFIAQVDLCVDCTTGCLKGLANVHCSDLLKKWLPDRCASFATKIQKQVDKIKGMAGD
KPTENNEDFNIVAQAMNFATTDLDADRGKLPGKKLPLEVLKEMEANARKAGCTRGCLICLSKIKCTLKMKKFIPGRCHTYEGDKSTGQGGIGPAIVDIPEIPGFKDMEPMEQFIAQVDLCEDCTTGCLKGLANVQCSDLLKKWLPQRCATFASKIQGQVDKIKGAGGD
KPTENNEDFNIVAVASNFATTDLDADRGKLPGKKLPLEVLKEMEANARKAGCTRGCLICLSKIKCTLKMKKFIPGRCHTYEGDKSTGQGGIGSAIVDIPEIPGFKDMEPMEQFIAQVDLCEDCTTGCLKGLANVQCSDLLK

# preparing for sits(positions) screening

In [9]:
import pandas as pd
import numpy as np
import re
# 位点筛选并生成突变矩阵
wt = "MKKYWNRRKNRVEDFINKLTSQLSKLFPDAIFIFEDLDKFNMYDKNSNFNRNLDRTNWRKIAKKLEYKSVVLYVNPHYTSKTCPVCGSKMKSQEGQVVKCDKCGIFDRQFVRCYNIFKRGVELAKKLLGGVGVPVAGAEVDDLLSNEPRGELRLVKPNPNVEAKLPVRKSNRRFELQNPKDFVQIFDFPLMVYTVDLNGKYLKIYNCP"
df1 = pd.read_csv(r"/mnt/dk/EvoPlay-main/data/design_preprocess_data/tnpb_fitness.csv") 



s_dict = {}
seq_list = []
fit_list = []
seq_list.append(wt)
seq_list.extend(df1["mutants"].tolist())
fit_list.append(1.)
fit_list.extend(df1["fitness_relative"].tolist())
assert len(fit_list)==len(seq_list)
s_dict["seqs"] = seq_list
s_dict["fitness"] = fit_list
for i in range(len(wt)):
    tmp_list = []
    tmp_list.append(wt[i])
    for j in range(len(df1)):
        tmp_list.append(df1.iloc[j]["mutants"][i])
    s_dict["{}".format(i)]=tmp_list
df_w = pd.DataFrame(s_dict)
df_w.to_csv(r"/mnt/dk/EvoPlay-main/data/design_preprocess_data/tnpb_pos_selec_2.csv", index=False)


# mutation space and available mutating moves

In [10]:
import pandas as pd
import numpy as np
import re
# 定义突变空间和可用的突变操作，并生成文件 pos_space_new_wt.csv，
# 该文件记录了每个氨基酸在酶序列不同位置上的突变频率和可行性。
# 这些信息将用于强化学习算法中，指导算法进行合理的突变操作
wt = "MKKYWNRRKNRVEDFINKLTSQLSKLFPDAIFIFEDLDKFNMYDKNSNFNRNLDRTNWRKIAKKLEYKSVVLYVNPHYTSKTCPVCGSKMKSQEGQVVKCDKCGIFDRQFVRCYNIFKRGVELAKKLLGGVGVPVAGAEVDDLLSNEPRGELRLVKPNPNVEAKLPVRKSNRRFELQNPKDFVQIFDFPLMVYTVDLNGKYLKIYNCP"
AAS = "ILVAGMFYWEDQNHCRKSTP"
# 创建氨基酸索引字典
# aas_d0：将氨基酸的索引编号和对应的氨基酸字符建立映射关系。
# aas_d：将氨基酸字符作为键，索引作为值，方便后续查询和操作。
aas_d0 = dict(list(enumerate(AAS)))
aas_d = {}
for k,v in aas_d0.items():
    aas_d[v] = k
print(aas_d)

# 加载之前处理过的突变体数据文件 pos_selec_round1_new.csv。
# 这个文件应该包含突变后的酶序列。
# seq_list：从文件中提取所有突变后的序列列表
df1 = pd.read_csv(r"/mnt/dk/EvoPlay-main/data/design_preprocess_data/tnpb_pos_selec_2.csv")
aa_list = list(AAS)

seq_list = df1["seqs"].tolist()


# 计算每个位置的突变频率 遍历突变体序列，计算每个位置上突变发生的次数，并存储在 count 数组中
count = np.zeros(len(wt))
for seq in seq_list:
    for i in range(len(wt)):
        if seq[i]!=wt[i]:
            count[i] +=1
print(count)

# 筛选发生突变的位置 筛选出那些发生了突变的位置，保存为pos_1_new.npy 文件。
# ind 记录了所有发生突变的位点索引。
arr_index = np.where(count!=0)
print(arr_index)
np.save(r"/mnt/dk/EvoPlay-main/data/design_preprocess_data/tnpb_pos_1_new.npy",arr_index)

ind = arr_index[0]+1


# 生成突变矩阵 为每个发生突变的位点创建一个数组 tmp_ar，记录每个氨基酸在该位置出现的次数。
# 最后，为每个位置保存一个突变矩阵，矩阵中的数值代表了该位置上每种氨基酸的替代频率。
s_dict = {}
s_dict["AA"] = list(AAS)
for i in ind:
    tmp_ar = np.zeros(len(AAS))
    for seq in seq_list:
        if seq[i-1]!=wt[i-1]:

            try:
                tmp_ar[aas_d[seq[i - 1]]]+=1
            except:
                print(seq)
    #加上wild_type   
    tmp_ar[aas_d[wt[i-1]]]+=1
    # 加上wild_type

    s_dict["{}".format(i)] = tmp_ar.tolist()
df_w = pd.DataFrame(s_dict)
df_w.to_csv(r"/mnt/dk/EvoPlay-main/data/design_preprocess_data/tnpb_pos_space_new_wt.csv", index = False)  #policy: available moves limitation




{'I': 0, 'L': 1, 'V': 2, 'A': 3, 'G': 4, 'M': 5, 'F': 6, 'Y': 7, 'W': 8, 'E': 9, 'D': 10, 'Q': 11, 'N': 12, 'H': 13, 'C': 14, 'R': 15, 'K': 16, 'S': 17, 'T': 18, 'P': 19}
[ 0.  0.  5.  0. 14.  0.  7.  0.  0.  0.  0.  2.  0.  0.  2.  1.  5.  0.
  3.  0.  0. 13.  0.  5.  0.  0.  5.  0.  0.  0.  0.  5.  0.  7.  0.  3.
  0.  7.  0.  0.  0.  0.  0.  2.  0.  0.  0.  3.  0.  0.  0.  2.  6.  0.
  0.  6.  0. 10.  5.  0.  2.  4.  0.  9.  5.  0. 17.  0.  0.  0.  0.  0.
  0.  0.  0. 11.  0.  0.  0.  0.  4.  0.  0.  0.  0.  0.  0.  7.  0.  0.
  0.  0.  0.  0.  0.  4.  0. 19. 11.  0. 14.  0.  0.  0. 13.  6. 10.  0.
  9.  0. 11.  8.  0.  9.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 12.
  0. 13.  9.  8.  0.  0.  0.  0.  5. 10.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  9. 10. 11.  0.  4.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  

# generate model input data

In [11]:
import pandas as pd
import numpy as np
import re

# 生成模型输入数据，并将特定位置的氨基酸序列及其对应的适应度值格式化为模型可用的输入文件
wt = "MKKYWNRRKNRVEDFINKLTSQLSKLFPDAIFIFEDLDKFNMYDKNSNFNRNLDRTNWRKIAKKLEYKSVVLYVNPHYTSKTCPVCGSKMKSQEGQVVKCDKCGIFDRQFVRCYNIFKRGVELAKKLLGGVGVPVAGAEVDDLLSNEPRGELRLVKPNPNVEAKLPVRKSNRRFELQNPKDFVQIFDFPLMVYTVDLNGKYLKIYNCP"
AAS = "ILVAGMFYWEDQNHCRKSTP"

df1 = pd.read_csv(r"/mnt/dk/EvoPlay-main/data/design_preprocess_data/tnpb_pos_selec_2.csv")
aa_list = list(AAS)

pos_arr = np.load(r"/mnt/dk/EvoPlay-main/data/design_preprocess_data/tnpb_pos_1_new.npy")


# 提取突变位点的氨基酸序列和适应度值
pos = pos_arr[0]
df_list = []
for i in range(len(df1)):

    seq_0 = df1.iloc[i]["seqs"]
    seq_list = [seq_0[p] for p in pos]
    seq = "".join(seq_list)
    df_list.append({"seqs":seq, "fitness":df1.iloc[i]["fitness"]})

df_w = pd.DataFrame(df_list)
# 这个文件将包含模型训练所需的输入数据，每一行记录突变体的序列和适应度值。
df_w.to_csv(r"/mnt/dk/EvoPlay-main/data/design_preprocess_data/tnpb_model_input_data_new.csv", index=False)
 


In [12]:
# 打印突变位点的索引，以及将这些位点转换为字符串格式，便于后续分析或记录。
arr_i = np.load(r"/mnt/dk/EvoPlay-main/data/design_preprocess_data/tnpb_pos_1_new.npy")
print(arr_i)
print(arr_i[0]+1)
arr_j = arr_i[0]+1
a_str_list = []
for a in arr_j:
    a_str_list.append(str(a))
print(",".join(a_str_list))
print(len(arr_i[0]))

[[  2   4   6  11  14  15  16  18  21  23  26  31  33  35  37  43  47  51
   52  55  57  58  60  61  63  64  66  75  80  87  95  97  98 100 104 105
  106 108 110 111 113 125 127 128 129 134 135 150 151 152 154]]
[  3   5   7  12  15  16  17  19  22  24  27  32  34  36  38  44  48  52
  53  56  58  59  61  62  64  65  67  76  81  88  96  98  99 101 105 106
 107 109 111 112 114 126 128 129 130 135 136 151 152 153 155]
3,5,7,12,15,16,17,19,22,24,27,32,34,36,38,44,48,52,53,56,58,59,61,62,64,65,67,76,81,88,96,98,99,101,105,106,107,109,111,112,114,126,128,129,130,135,136,151,152,153,155
51
